In [1]:
import numpy as np;
import pandas as pd;

In [54]:
ROOT_DIR = "/Users/sreeramh/Documents/Kerala_Covid/SpatioTemporal/gitsrc/SpatioTemporalSEIR"
mu = 0.01

```
                P1
               /. \
     ----------    -------
     |                   |
     10                 10
     |                   |
     P1.1 ---- 10 ----- P1.2           
     / \                / \
    10  10            10  10 
   /      \          /      \
  P1.1.1  P1.1.2  P1.2.1.  P1.2.2 
```

In [65]:
initDataDF = pd.read_csv(ROOT_DIR + "/data/sample_init_data.csv")

distanceData = pd.read_csv(ROOT_DIR + "/data/sample_distance_matrix.csv")

r = len(initDataDF.index)

In [66]:
distanceMatrix = distanceData['value'].to_numpy().reshape(r,r)
distanceMatrix

array([[ 0, 10, 10, 20, 20, 20, 20],
       [10,  0, 10, 10, 10, 20, 20],
       [10, 10,  0, 20, 20, 10, 10],
       [20, 10, 20,  0, 20, 30, 30],
       [20, 10, 20, 20,  0, 30, 30],
       [20, 20, 10, 30, 30,  0, 20],
       [20, 20, 10, 30, 30, 20,  0]])

In [67]:
def getSij(i, j, distanceMatrix, initDataDF):
    return(initDataDF[distanceMatrix[i] <= distanceMatrix[j]]['S'].sum())
    


In [49]:
getSij(3,1,distanceMatrix, initDataDF)

7500

In [68]:
def getTij(i,j, distanceMatrix, initDataDF):
    Ni = initDataDF.iloc[i]['S']
    Ti = mu*Ni
    if(i==j):
        return Ni - Ti;
    
    Nj = initDataDF.iloc[j]['S']
    Sij = getSij(i,j,distanceMatrix, initDataDF)
    Tij = Ti * ((Ni * Nj) /((Ni + Sij)*(Ni + Nj + Sij)))
    return Tij
    
    

In [77]:
resultsDF = pd.DataFrame(columns=['iName', 'jName', 'Tij'], );
for i in range(0,r-1):
    for j in range(0,r-1):
        Tij = getTij(i,j,distanceMatrix, initDataDF)
        resultsDF = resultsDF.append({'iName': initDataDF.iloc[i]['name'], 'jName': initDataDF.iloc[j]['name'], 'Tij': Tij},ignore_index=True)
 

In [78]:
resultsDF.to_csv(ROOT_DIR + "/output/tmatrix_results.csv")